<a href="https://colab.research.google.com/github/FaddliLWibowo/-Twitter-Sentiment-Analysis-with-Streamlit/blob/main/Data_Collection_From_Twitter_For_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor==0.6.0
!pip install tweepy==4.10.1
!pip install textblob==0.15.3
!pip install wordcloud==1.8.2.2
!pip install nltk==3.7
!pip install googletrans==4.0.0-rc1

In [12]:
from wordcloud import WordCloud, STOPWORDS
from datetime import timedelta, datetime
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt 
from tweepy import OAuthHandler
from textblob import TextBlob
import preprocessor as p
import seaborn as sns
import pandas as pd
import numpy as np
import tweepy
import csv
import nltk
import re
import string 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#Masukkan twitter token API
api_key = "GbIGEYyXJW8RtX0KBxtzjNFWb"
api_key_secret = "q47qwDYnsI9I6LAbifsQq5xULG4gYmiaOXD6iZZIjdKeNLXPT5"
access_token = "847732131132358656-yvElNA1Ml5KM19Khl3CMNiMkb7Knk1Y"
access_token_secret ="17KXZyRBfXmDK8UfsMozBnt3RP2C5vA8Rt8vA5rFHoUHY"

In [14]:
# melakukan authentication atau registrasi api key ke api twitter
auth = OAuthHandler(api_key, api_key_secret)
# Mendapat akses dari token yang didapat
auth.set_access_token(access_token, access_token_secret)
# memanggil API twitter
api = tweepy.API(auth)

In [16]:
def scraptweets(search_words, date_since, date_until,  numTweets):

    db_tweets = pd.DataFrame(columns=['id', 'tweetcreatedts', 'username', 'text', 'source', 'like', 'retweet'])

    tweets = tweepy.Cursor(
                    api.search_tweets, q=search_words, lang="id", 
                    since=date_since, until=date_until,  tweet_mode='extended').items(numTweets)

    tweet_list = [tweet for tweet in tweets]

    for tweet in tweet_list:
        username = tweet.user.screen_name
        tweetcreatedts = tweet.created_at
        id_tweet = tweet.user.id
        source = tweet.source
        favorite_count = tweet.favorite_count
        retweet_count = tweet.retweet_count

        try:
            text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.retweeted_status.full_text).split())
        except AttributeError:
            text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.full_text).split())

        ith_tweet = [id_tweet, tweetcreatedts, username, text, source, favorite_count, retweet_count]

        db_tweets.loc[len(db_tweets)] = ith_tweet
    
    print('Proses Scrapping Selesai Dengan Jumlah Data', len(db_tweets))
    filename = 'Data-hasil-crawling01.csv'
    db_tweets.to_csv(filename, index=False)

today = datetime.today().strftime("%Y-%m-%d")
last_week = datetime.today() - timedelta(1)
last_week = last_week.strftime("%Y-%m-%d")

search_words = "#pemilu OR #pemilu2024 -filter:retweets"
date_since = last_week
date_until = today
numTweets = 500

scraptweets(search_words, date_since, date_until, numTweets)

TooManyRequests: ignored

In [ ]:
data = pd.read_csv('Data-hasil-crawling01.csv')
data.sample(n=5)

In [ ]:
data.isnull().sum()

In [ ]:
#Mengganti format tanggal
tanggal = pd.to_datetime(data['tweetcreatedts'])
data['tweetcreatedts'] = tanggal.dt.strftime('%d/%m/%Y %H:%M')
data['tweetcreatedts']

In [ ]:
#Mengecek informasi pada kolom dataframe
data.info()

In [ ]:
# Menampilkan semua tanda baca dalam bentuk string
string.punctuation

In [ ]:
# Menghilangkan tanda baca dan karakter yang tidak diperlukan
def remove_punct(Tweets):
      #Menggabungkan karakter pada tweet
      Tweets = "".join([char for char in Tweets if char not in string.punctuation])
      #Menghilangkan angka
      Tweets = re.sub('[0-9]+', '', Tweets)
      #Menghilangkan url web
      Tweets = re.sub(r"http\S+", "", Tweets)
      Tweets = re.sub(r"www.\S+", "", Tweets)
      #Mengganti tweet ke huruf kecil  
      Tweets = Tweets.lower()
      return Tweets

# Menggunakan function pada kolom tweet
data['text'] = data['text'].apply(lambda x: remove_punct(x))
data.head()

In [ ]:
#Menghilangkan duplikat pada tweet
data = data.reset_index(drop=True).drop_duplicates(subset='text', keep="last", ignore_index=True)
data

In [ ]:
plt.figure(figsize=(7,4))
sns.countplot(x=data.source, color='mediumseagreen')
plt.title('Class distribution')
plt.ylabel('Class Counts')
plt.xlabel('Class Label')
plt.xticks(rotation='vertical');

In [ ]:
def preprocessing_data(x):
    return p.clean(x)

def tokenize_data(x):
    return p.tokenize(x)

data['tweet_clean'] = data['text'].apply(preprocessing_data)
data['tweet_clean'] = data['tweet_clean'].apply(tokenize_data)
data = data.drop_duplicates()

In [ ]:
from googletrans import Translator
translates = Translator()

def convert_eng(tweet):
    return translates.translate(tweet, dest='en').text

data['tweet_english'] = data['tweet_clean'].apply(convert_eng)

In [ ]:
ps = PorterStemmer() 

def stemming_data(x):
    return ps.stem(x)

data['tweet_english'] = data['tweet_english'].apply(stemming_data)

In [ ]:
data_tweet = list(data['tweet_english'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

def klasifikasi(polaritas):
    if polaritas > 0.0:
        return 'Positif'
    elif polaritas > 0.0 == 0.0:
        return 'Netral'
    else:
        return 'Negatif'

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1 

status = pd.DataFrame({'klasifikasi': status})
data['klasifikasi'] = status
data.tail()

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

In [ ]:
data.to_excel('Data-hasil-klasifikasi.xlsx', index=False)

In [ ]:
def show_pie(label, data, legend_title) :
    fig, ax = plt.subplots(figsize=(8, 10), subplot_kw=dict(aspect='equal'))

    labels = [x.split()[-1] for x in label]

    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}% ({:d})".format(pct, absolute)

    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data), 
                                      textprops=dict(color="w"))

    ax.legend(wedges, labels,
              title= legend_title,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=10, weight="bold")
    plt.show()

In [ ]:
label = ['Positif', 'Negatif', 'Netral']
count_data = [total_positif+1, total_negatif+1, total_netral]

show_pie(label, count_data, "Status")

In [ ]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud) 
    plt.axis("off");

In [ ]:
all_words = ' '.join([tweets for tweets in data['tweet_clean']])
wordcloud = WordCloud(width = 3000, height = 2000, random_state=3, background_color='white', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(all_words)
plot_cloud(wordcloud)

In [ ]:
all_words = ' '.join([tweets for tweets in data['tweet_english']])
wordcloud = WordCloud(width = 3000, height = 2000, random_state=3, background_color='white', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(all_words)
plot_cloud(wordcloud)